In [2]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time


In [3]:
# setup basic arguments for qed-ci calculation
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}


cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}



In [4]:

N_el = 50
N_ph = 10
omega = 0.12086
lambda_vector = np.array([0., 0., 0.05])


E_array = np.zeros(N_el)
LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)

slow_start = time.time()
LiH.build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector)
slow_end = time.time()

slow_eigs = np.copy(LiH.PCQED_eigs)

fast_start = time.time()
LiH.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector)
fast_end = time.time()
fast_eigs = np.copy(LiH.PCQED_eigs)

assert np.allclose(slow_eigs, fast_eigs)
print(F"Slow build took {slow_end-slow_start} seconds")
print(F"Fast build took {fast_end-fast_start} seconds")





Start SCF iterations:

Canonical RHF One-electron energy = -12.2195250859902966
CQED-RHF One-electron energy      = -12.2195250859902966
Nuclear repulsion energy          = 0.8819620177833333
Dipole energy                     = 0.0000000000000000
SCF Iteration   1: Energy = -7.8500186970978643   dE = -7.85002E+00   dRMS = 5.02710E-15
SCF Iteration   2: Energy = -7.8500186970978643   dE =  0.00000E+00   dRMS = 1.21935E-15
Total time for SCF iterations: 0.000 seconds 

QED-RHF   energy: -7.85001870 hartree
Psi4  SCF energy: -7.85001870 hartree
 Completed QED-RHF in 0.35631680488586426 seconds
 Completed 1HSO Build in 8.0108642578125e-05 seconds
 Completed ERI Build in 0.0012390613555908203 seconds 
 Completed 2D build in 0.00010585784912109375 seconds
 Completed 1G build in 1.6927719116210938e-05 seconds
 Completed the Dipole Matrix Build in 4.601478576660156e-05 seconds
 Completed determinant list in 0.0004999637603759766 seconds 
 Completed constant offset matrix in 0.0002160072326660

NameError: name 'lam_array' is not defined